### General Approach:

- We can solve for the binomial CDF which tells us likelihood of getting up to `x` successes (`P(X <= x)`) out of `n` attempts with likelihood `p`
    - Scipy's binom.cdf func is `P(X <= x)` so will need to solve for `k - 1`
    - For our problem, we want likelihood of `k` through `n` heads out of `n` remaining trials. This means we can take 1 - CDF(`k-1`,`n`,0.5)
    
    - Tangible example: 
        - We have run 15 trials, with 8 heads thus far.
		  - 51 - 8 = 43 heads left
		  - We can solve for 1 - (X <= 42), which tells us likelihood of getting > 42 heads (43+)
          - *In scipy, this will need to be written as 1 - (X <= x), where x is 43 - 1*

- We only count a trial when p > 0.99, then determine if person lost or not. 

#### let's say we have 12 flips left & we need 4 flips or more to break 51. 
- What we want is `P(X >= x)` where x = 4. 
    - `P(X >= x)` = `1 - P(X < x)`, so we can just solve 1 - CDF(x - 1,flips_left,0.5)
    - `P(X >= 4) = 0.92700195`

```python
p = 1 - binom.cdf(3,12,0.5) # 0.927001953125
```

#### Speeding things up:
- My initial runs were calculating the CDF 101 times per iteration. So my 100K test loop was doing many CDF calls  (100K & 101 = `10,100,000`)
- Given there are only so many combinations of `k` & `n` for cdf, can solve for each of those and then pass in as a dictionary. 
    - Note: Solving for `1 - binom.cdf(k - 1, n,  0.5)`
- A single run speeds up from `11.5 ms` to `0.338 ms` with this switch - about a 30x speedup. 

In [4]:
import numpy as np 
from scipy.stats import binom

In [7]:
# build cdf dictionary for possible turns left, heads we need.
# max heads is 51, max turns left is 101
# store off the actual p value to reduce operations 
tot_turns = 101
max_heads = 51
bin_cdf = {}

for n in range(tot_turns+1):
    for k in range(max_heads + 1):
        
        # using k-1 still .cdf func seems to be <= x
        bin_cdf[k,n] = 1 - binom.cdf(k-1,n,0.5)

In [8]:
# check a few: https://stattrek.com/online-calculator/binomial.aspx
# 1) P(X >= 10) when we have 47 flips left is 0.9999875479846878
print(bin_cdf.get((10,47))) # pretty good matching

# 2) P(X >= 30) when we have 47 flips left is 0.03947034684
print(bin_cdf.get((30,47))) # pretty good matching

0.9999875479846878
0.03947034684074424


In [9]:
def faster_game():
    
    # General game params:
    heads = 0
    winning_flips = 51
    tot_flips = 101
    flips = 0
    exceeds = False
    winner = False

    # Run 101 flips. If a clear winner arises, end game
    for _ in range(tot_flips):

        # flip coin once
        coin = np.random.binomial(1,0.5)

        # update heads count, flips, and tails 
        heads += coin
        flips += 1
        tails = flips - heads   
        
        # check if game is over for either person
        if heads >= winning_flips:
            winner = True

        # needed to win:
        needed_heads = winning_flips - heads
        turns_left = tot_flips - flips
        
        #### STOP GAME IF WINNER IS FOUND: 
        
        ### Solve for likelihood of getting needed_heads or more based on remaining turns
        k = max(0, needed_heads) # whats the likelihood of getting up to needed heads but not over; we can't find negatives, would just be 100%
        n = turns_left

        # binom.cdf calculates P(X <= k)
        p = bin_cdf.get((k,n))
        if p >= 0.99:
            exceeds = True
            
    return exceeds, winner

### Rerun with New Function: 30x faster

- initially misread the question, but we want to know `P(p >= 0.99 & lose)`, I was solving for `P(Lose | p  >= 0.99)`

In [14]:
tot_games = 0
super_loser = 0

for j in range(2000000):
    
    exceeds, winner = faster_game()
    
    tot_games += 1
    
    # out of loop
    if exceeds and winner == False:
        super_loser += 1
     
    if (j+1) % 200000 == 0:
        print(f"Iteration {j+1}:")
        print(f"\tTotal games: {tot_games}")
        print(f"\tSuper loser games: {super_loser}")
        print(f"\tPercent: {100 * super_loser / tot_games:.6f}")

Iteration 200000:
	Total games: 200000
	Super loser games: 421
	Percent: 0.210500
Iteration 400000:
	Total games: 400000
	Super loser games: 817
	Percent: 0.204250
Iteration 600000:
	Total games: 600000
	Super loser games: 1261
	Percent: 0.210167
Iteration 800000:
	Total games: 800000
	Super loser games: 1700
	Percent: 0.212500
Iteration 1000000:
	Total games: 1000000
	Super loser games: 2097
	Percent: 0.209700
Iteration 1200000:
	Total games: 1200000
	Super loser games: 2504
	Percent: 0.208667
Iteration 1400000:
	Total games: 1400000
	Super loser games: 2943
	Percent: 0.210214
Iteration 1600000:
	Total games: 1600000
	Super loser games: 3321
	Percent: 0.207563
Iteration 1800000:
	Total games: 1800000
	Super loser games: 3737
	Percent: 0.207611
Iteration 2000000:
	Total games: 2000000
	Super loser games: 4178
	Percent: 0.208900
